In [15]:
import gym
import numpy as np
from math import sqrt
import time
import random

In [60]:
env = gym.make('CartPole-v0')
env.reset()

exploration = 0.9
NEG_INF = -99999999999



observation,reward,done,_ = env.step(env.action_space.sample()) # take a random action

# print(observation)


# print(discretize_observation(observation))

# print(env.action_space.start)

# for i in range(env.action_space.start,env.action_space.n):
#     print(i)

env.close()



## Getting Discrete state

In [55]:
# n_steps = np.array([100,100,100,100])
n_steps = 50

env_low = np.float64(env.observation_space.low)
env_high = np.float64(env.observation_space.high)

step = (env_high-env_low)/n_steps

def discretize_observation(observation):
    return ((observation - env_low) / step).astype(np.int8)


# obs = env.reset()

# while True:
#     obs,_,done,_ = env.step(env.action_space.sample())
#     print(discretize_observation(obs))
    
#     if done:
#         break

## Defining Policy and Value

In [40]:

## Dict where key is discrete_state + action
Q_values = {}


def get_state_action(obs,action):
    if(np.dtype(obs[0]) is not np.int8):
        obs = discretize_observation(obs)
    
    sa = (tuple(obs),action)
    return sa

def get_state(obs):
    if(np.dtype(obs[0]) is not np.int8):
        obs = discretize_observation(obs)
    
    sa = tuple(obs)
    return sa

def policy(obs):
    # add discrete check
    if(np.dtype(obs[0]) is not np.int8):
        obs = discretize_observation(obs)
    
    t = random.random()
    
    if t < exploration:
        act = env.action_space.sample()
        qa = get_state_action(obs,act)
        
        if qa not in Q_values:
            Q_values[qa] = 0
        
        return act
    
    else:
        max_val = NEG_INF
        max_i = 0
        
        for act in range(env.action_space.n):
            qa = get_state_action(obs,act)

            if qa not in Q_values:
                Q_values[qa] = 0

            if Q_values[qa] > max_val:
                max_val = Q_values[qa]
                max_i = act

        return max_i



## Running a episode

In [41]:

def run_episode(render = False):
    vals = []
    obs = env.reset()
    done = False
    while not done:
        ## one episode has obs, action, reward
        action = policy(obs)
        ep_val = [discretize_observation(obs),action]
        
        if render:
            env.render()
        
        obs,reward,done,_ = env.step(action)
        ep_val.append(reward)
        vals.append(ep_val.copy())
        
    return vals

print(len(run_episode()))

15


## Update Q table

In [47]:

def update_Q(render=False):
    ep = run_episode(render)
    reward_sum = 0
    explored = set()
        
    for i in reversed(ep):
        s = get_state(i[0])
        sa = get_state_action(i[0],i[1])
        
        if s in explored:
            continue
        explored.add(s)
        
        reward_sum = reward_sum + i[2]
        
         
        if sa not in Q_values:
            Q_values[sa] = 0
            
        Q_values[sa] = Q_values[sa] + (reward_sum - Q_values[sa])
    
    return reward_sum

    


In [62]:
Q_values = {}


for i in range(100000):
    total_reward = update_Q()
    exploration = exploration * 0.95
    if i%5000 == 0:
        print("Total Reward is ",total_reward)
    


Total Reward is  8.0
Total Reward is  11.0
Total Reward is  10.0
Total Reward is  8.0
Total Reward is  9.0
Total Reward is  10.0
Total Reward is  10.0
Total Reward is  7.0
Total Reward is  7.0
Total Reward is  10.0
Total Reward is  10.0
Total Reward is  7.0
Total Reward is  10.0


KeyboardInterrupt: 

In [63]:
print(exploration)

4.4e-323
